In [ ]:
from netsuitesdk import NetSuiteConnection

import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [ ]:
NETSUITE_CONFIG = {
    'account': os.getenv('account'),
    'consumer_key': os.getenv('consumer_key'),
    'consumer_secret': os.getenv('consumer_secret'),
    'token_key': os.getenv('token_key'),
    'token_secret': os.getenv('token_secret'),
    'wsdl_version': os.getenv('wsdl_version')
}

In [ ]:
ns = NetSuiteConnection(
    account=NETSUITE_CONFIG['account'],
    consumer_key=NETSUITE_CONFIG['consumer_key'],
    consumer_secret=NETSUITE_CONFIG['consumer_secret'],
    token_key=NETSUITE_CONFIG['token_key'],
    token_secret=NETSUITE_CONFIG['token_secret']

)

In [ ]:
count = 0
for attr in dir(ns):
    try:
        obj = getattr(ns, attr)
        if hasattr(obj, "get_all"):
            print(f"{attr}")
            count += 1
    except Exception:
        pass
    print(f"\n Total accessible objects: {count}")

In [ ]:
for attr in dir(ns):
    try:
        obj = getattr(ns, attr)
        if hasattr(obj, "get_all"):
            records = obj.get_all()
            print(f"{attr:25}: {len(records)} records")
    except Exception as e:
        print(f"{attr:25}: Error {e}")

# method

In [ ]:
methods = [m for m in dir(ns.accounts) if not m.startswith('_')]
print("Invoice methods:", methods)

In [ ]:
import itertools
all_accounts = list(itertools.islice(ns.accounts.get_all_generator(), 1000))

In [ ]:
len(all_accounts[0])

In [ ]:
res = ns.vendor_payments.get_all_generator()
res

In [ ]:
len(res)

In [ ]:
PAGE_SIZE = 500

def get_record_count(obj):

    if hasattr(obj, 'create_paginated_search'):
        try:
            pages = obj.create_paginated_search(PAGE_SIZE)
            total = 0
            for page in pages:
                total += len(page)
            return total
        except Exception as e:
            print(f"Error creating paginated search for {obj}: {e}")  



In [ ]:
import itertools

PAGE_SIZE = 500

def get_record_count(obj):

    if hasattr(obj, 'create_paginated_search'):
        try:
            pages = obj.create_paginated_search(PAGE_SIZE)
            total = 0
            for page in pages:
                total += len(page)
            return total
        except Exception as e:
            print(f"Error creating paginated search for {obj}: {e}")            
print("\n Record count per accessible object:\n")
for attr in dir(ns):
    if attr.startswith('_'):
        continue
    obj = getattr(ns, attr)
    print(f"{attr:25}: {get_record_count(obj)}")


In [ ]:
import time
import tracemalloc

record_types = ['vendor_payments']


start_time = time.time()
tracemalloc.start()

for record_type in record_types:
    try:
        obj = getattr(ns, record_type)
        print(f"Fetching {record_type}")
        records = obj.get_all()
        print(f" Total records: {len(records)}")
    except Exception as e:
        print(f"Error fetching {record_type}: {e}")

end_time = time.time()
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()

print(f"\n Current Memory Usage: {current / (1024 * 1024):.2f} MB")
print(f"Peak Memory Usage: {peak / (1024 * 1024):.2f} MB")
print(f"Total Time: {end_time - start_time:.2f} seconds")


In [ ]:

vendor_bills = ns.vendor_payments.get_all()


In [ ]:
vendor_bills

In [ ]:
vendor_payments= list(itertools.islice(ns.vendor_payments.get_all_generator(), 1))

In [ ]:
adv=ns.adv_inter_company_journal_entries.get_all_generator()

In [ ]:
vendor_payments

netsuite dict to dataframe

In [ ]:
from zeep.helpers import serialize_object
import pandas as pd

df = pd.json_normalize([serialize_object(obj) for obj in vendor_payments], sep='_')

In [ ]:
df

In [ ]:
print(dir(ns)) 

In [ ]:
def get_all_vendor_payments_with_pagination(ns, batch_size=1000, max_records=None):
    """
    Retrieve all vendor payments using pagination with generator
    """
    vendor_payments = []
    count = 0
    print(f"Starting to retrieve vendor payments with batch size: {batch_size}")
    try:

        for payment in ns.adv_inter_company_journal_entries.get_all_generator():
            vendor_payments.append(payment)
            count += 1
            if count % 1000 == 0:
                print(f"Retrieved {count} vendor payments.")
            if max_records and count >= max_records:
                break
        print(f"Total vendor payments retrieved: {count}")
        return vendor_payments
        
    except Exception as e:
        print(f"Error retrieving vendor payments: {e}")
        return vendor_payments

In [ ]:
# Method 1: Get all vendor payments with pagination
print("=== Method 1: Get all vendor payments with pagination ===")
vendor_payments = get_all_vendor_payments_with_pagination(ns, batch_size=1000, max_records=7000)
print(f"Retrieved {len(vendor_payments)} vendor payments")

In [ ]:
from zeep.helpers import serialize_object
import pandas as pd

df = pd.json_normalize([serialize_object(obj) for obj in batch], sep='_')

In [ ]:
df